# Puzzle #120 - GPU 1-Hour Run

## Overview

This notebook is configured to run Kangaroo ECDLP solver on **Puzzle #120** using **GPU acceleration** for exactly **1 hour**.

### Puzzle #120 Specifications

- **Address**: 17s2b9ksz5y7abUm92cHwG8jEPCzK3dLnT
- **Range Start**: 800000000000000000000000000000 (hex)
- **Range End**: FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF (hex)
- **Public Key**: 02CEB6CBBCDBDF5EF7150682150F4CE2C6F4807B349827DCDBDD1F2EFA885A2630
- **Range Width**: ~119 bits
- **Expected Operations**: ~2^60.5 (highly dependent on hardware)

### Important Notes

⚠️ **Warning**: Puzzle #120 is extremely difficult and cannot be solved in 1 hour even with powerful GPUs. This notebook is designed for:
- Testing GPU performance on a challenging puzzle
- Collecting work progress that can be resumed later
- Benchmarking your hardware capabilities
- Contributing to distributed solving efforts

The work file will be saved periodically and can be resumed in future runs or merged with other work files.

## Section 1: Environment Setup

### Check GPU Availability

In [ ]:
# Check for NVIDIA GPU and CUDA installation
import os
import subprocess

print("=" * 60)
print("GPU ENVIRONMENT CHECK")
print("=" * 60)

# Check nvidia-smi
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print("\n✅ NVIDIA GPU detected!\n")
        print(result.stdout)
    else:
        print("\n❌ nvidia-smi failed. No GPU detected.")
except FileNotFoundError:
    print("\n❌ nvidia-smi not found. NVIDIA drivers may not be installed.")
except Exception as e:
    print(f"\n❌ Error checking GPU: {e}")

# Check CUDA
print("\n" + "=" * 60)
print("CUDA TOOLKIT CHECK")
print("=" * 60)
try:
    result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print("\n✅ CUDA compiler detected!\n")
        print(result.stdout)
    else:
        print("\n⚠️  CUDA compiler not available. Will attempt to use system CUDA.")
except FileNotFoundError:
    print("\n⚠️  nvcc not in PATH. Will attempt to use system CUDA at /usr/local/cuda")
except Exception as e:
    print(f"\n⚠️  Error checking CUDA: {e}")

print("\n" + "=" * 60)

### Clone and Navigate to Repository

If running this notebook standalone, clone the Kangaroo repository first.

In [ ]:
import os

# Check if we're already in the kangaroo directory
if not os.path.exists('kangaroo_ecdlp_solver.ipynb'):
    print("Cloning Kangaroo repository...")
    !git clone https://github.com/drqsatoshi/kangaroo.git
    %cd kangaroo
else:
    print("Already in kangaroo directory")

# Show current directory
!pwd
print("\nRepository files:")
!ls -la | head -20

## Section 2: Build Kangaroo with GPU Support

### Determine GPU Compute Capability

In [ ]:
# Try to detect GPU compute capability
import subprocess
import re

ccap = "75"  # Default to 7.5 (Tesla T4, RTX 2080 Ti)

try:
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=name,compute_cap', '--format=csv,noheader'],
        capture_output=True, text=True, timeout=5
    )
    if result.returncode == 0:
        lines = result.stdout.strip().split('\n')
        if lines:
            gpu_info = lines[0]  # First GPU
            print(f"Detected GPU: {gpu_info}")
            # Extract compute capability
            match = re.search(r'(\d+)\.(\d+)', gpu_info)
            if match:
                ccap = match.group(1) + match.group(2)
                print(f"Compute Capability: {match.group(1)}.{match.group(2)} (ccap={ccap})")
except Exception as e:
    print(f"Could not auto-detect compute capability: {e}")
    print(f"Using default ccap={ccap}")

print(f"\n✅ Will compile with ccap={ccap}")
print("\nCommon GPU Compute Capabilities:")
print("  Tesla K80:    3.7 (ccap=37)")
print("  Tesla V100:   7.0 (ccap=70)")
print("  Tesla T4:     7.5 (ccap=75)")
print("  RTX 2080 Ti:  7.5 (ccap=75)")
print("  Tesla A100:   8.0 (ccap=80)")
print("  RTX 3090:     8.6 (ccap=86)")
print("  RTX 4090:     8.9 (ccap=89)")
print("  Tesla L4:     8.9 (ccap=89)")

### Compile with GPU Support

In [ ]:
# Clean previous builds
print("Cleaning previous builds...")
!make clean

# Build with GPU support
print(f"\nBuilding Kangaroo with GPU support (ccap={ccap})...")
print("This may take several minutes...\n")

# Try to build - adjust CUDA path as needed
!make gpu=1 CUDA=/usr/local/cuda ccap={ccap} all -j4

# Check if build succeeded
import os
if os.path.exists('./kangaroo'):
    print("\n" + "=" * 60)
    print("✅ BUILD SUCCESSFUL!")
    print("=" * 60)
    !./kangaroo -v
else:
    print("\n" + "=" * 60)
    print("❌ BUILD FAILED!")
    print("=" * 60)
    print("Please check the error messages above.")

## Section 3: Verify Puzzle #120 Input File

The repository should already contain `puzzle120.txt`. Let's verify it has the correct configuration.

In [ ]:
# Check if puzzle120.txt exists
import os

if os.path.exists('puzzle120.txt'):
    print("✅ puzzle120.txt found!\n")
    print("Content:")
    !cat puzzle120.txt
else:
    print("⚠️  puzzle120.txt not found. Creating it...\n")
    # Create the file with correct values
    with open('puzzle120.txt', 'w') as f:
        f.write("800000000000000000000000000000\n")
        f.write("FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF\n")
        f.write("02CEB6CBBCDBDF5EF7150682150F4CE2C6F4807B349827DCDBDD1F2EFA885A2630\n")
    print("✅ Created puzzle120.txt")
    !cat puzzle120.txt

print("\n" + "=" * 60)
print("PUZZLE #120 DETAILS")
print("=" * 60)
print("Address:     17s2b9ksz5y7abUm92cHwG8jEPCzK3dLnT")
print("Start Range: 800000000000000000000000000000 (hex)")
print("End Range:   FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF (hex)")
print("Public Key:  02CEB6CBBCDBDF5EF7150682150F4CE2C6F4807B349827DCDBDD1F2EFA885A2630")
print("Range Width: ~119 bits")
print("Difficulty:  Extremely High (requires months/years on powerful hardware)")
print("=" * 60)

## Section 4: Configure Run Parameters

Set up the parameters for a 1-hour GPU run with work file saving.

In [ ]:
import time
from datetime import datetime

# Configuration
RUN_TIME_SECONDS = 3600  # 1 hour
WORK_FILE = f"puzzle120_work_{datetime.now().strftime('%Y%m%d_%H%M%S')}.work"
SAVE_INTERVAL = 300  # Save work every 5 minutes (300 seconds)
OUTPUT_FILE = f"puzzle120_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
DP_BITS = 25  # Distinguished point bits (recommended for puzzle #120)
GPU_IDS = "0"  # Use first GPU (modify if you want to use multiple GPUs)

print("=" * 60)
print("RUN CONFIGURATION")
print("=" * 60)
print(f"Run Duration:          {RUN_TIME_SECONDS} seconds (1 hour)")
print(f"Work File:             {WORK_FILE}")
print(f"Save Interval:         {SAVE_INTERVAL} seconds")
print(f"Output File:           {OUTPUT_FILE}")
print(f"Distinguished Bits:    {DP_BITS}")
print(f"GPU IDs:               {GPU_IDS}")
print(f"Input File:            puzzle120.txt")
print("=" * 60)
print("\nℹ️  The work file will be saved automatically every 5 minutes.")
print("ℹ️  You can resume this work later using the -i option.")
print("ℹ️  Multiple work files can be merged to combine progress.")
print("=" * 60)

## Section 5: Run Kangaroo on GPU for 1 Hour

This will start the GPU solver and automatically terminate after 1 hour.

In [ ]:
import subprocess
import time
import signal
from datetime import datetime, timedelta

# Build the command
cmd = [
    './kangaroo',
    '-gpu',
    '-gpuId', GPU_IDS,
    '-d', str(DP_BITS),
    '-ws',  # Save kangaroos in work file
    '-w', WORK_FILE,
    '-wi', str(SAVE_INTERVAL),
    '-o', OUTPUT_FILE,
    'puzzle120.txt'
]

print("=" * 60)
print("STARTING KANGAROO GPU SOLVER")
print("=" * 60)
print(f"Start Time:  {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"End Time:    {(datetime.now() + timedelta(seconds=RUN_TIME_SECONDS)).strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Duration:    {RUN_TIME_SECONDS} seconds (1 hour)")
print("=" * 60)
print(f"\nCommand: {' '.join(cmd)}\n")
print("=" * 60)
print("OUTPUT FROM KANGAROO")
print("=" * 60)
print()

# Start the process
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    universal_newlines=True
)

start_time = time.time()
last_output_time = start_time

try:
    # Monitor the process for exactly 1 hour
    while True:
        current_time = time.time()
        elapsed = current_time - start_time
        
        # Check if 1 hour has passed
        if elapsed >= RUN_TIME_SECONDS:
            print("\n" + "=" * 60)
            print("⏰ 1 HOUR COMPLETED - TERMINATING PROCESS")
            print("=" * 60)
            process.terminate()
            time.sleep(2)
            if process.poll() is None:
                process.kill()
            break
        
        # Check for output
        line = process.stdout.readline()
        if line:
            print(line, end='')
            last_output_time = current_time
        
        # Check if process has ended
        if process.poll() is not None:
            # Process ended (possibly found the key!)
            remaining_output = process.stdout.read()
            if remaining_output:
                print(remaining_output, end='')
            print("\n" + "=" * 60)
            print("🎉 PROCESS COMPLETED BEFORE 1 HOUR!")
            print("=" * 60)
            break
        
        # Small delay to prevent busy waiting
        time.sleep(0.1)

except KeyboardInterrupt:
    print("\n" + "=" * 60)
    print("⚠️  INTERRUPTED BY USER")
    print("=" * 60)
    process.terminate()
    time.sleep(2)
    if process.poll() is None:
        process.kill()

finally:
    # Wait for process to finish
    process.wait()
    
    end_time = time.time()
    total_time = end_time - start_time
    
    print("\n" + "=" * 60)
    print("RUN SUMMARY")
    print("=" * 60)
    print(f"Total Runtime:    {total_time:.1f} seconds ({total_time/60:.1f} minutes)")
    print(f"Target Runtime:   {RUN_TIME_SECONDS} seconds (1 hour)")
    print(f"Work File:        {WORK_FILE}")
    print(f"Output File:      {OUTPUT_FILE}")
    print("=" * 60)

## Section 6: Check Results and Work File

After the run completes, let's examine what was accomplished.

In [ ]:
import os

print("=" * 60)
print("POST-RUN ANALYSIS")
print("=" * 60)

# Check for output file (would exist if key was found)
if os.path.exists(OUTPUT_FILE):
    print("\n🎉 OUTPUT FILE FOUND - KEY MAY HAVE BEEN SOLVED!\n")
    print(f"Contents of {OUTPUT_FILE}:")
    print("=" * 60)
    !cat {OUTPUT_FILE}
    print("=" * 60)
else:
    print(f"\nℹ️  No output file found ({OUTPUT_FILE})")
    print("   This is expected - puzzle #120 requires much more time.")

# Check for work file
print("\n" + "=" * 60)
print("WORK FILE STATUS")
print("=" * 60)

if os.path.exists(WORK_FILE):
    file_size = os.path.getsize(WORK_FILE)
    print(f"\n✅ Work file saved: {WORK_FILE}")
    print(f"   Size: {file_size:,} bytes ({file_size/1024/1024:.2f} MB)")
    print("\nℹ️  This work file contains your progress and can be:")
    print("   1. Resumed using: ./kangaroo -i", WORK_FILE)
    print("   2. Merged with other work files")
    print("   3. Inspected using: ./kangaroo -winfo", WORK_FILE)
else:
    print(f"\n⚠️  Work file not found: {WORK_FILE}")
    print("   The process may not have run long enough to save.")

# List all work files in current directory
print("\n" + "=" * 60)
print("ALL WORK FILES IN DIRECTORY")
print("=" * 60)
!ls -lh *.work 2>/dev/null || echo "No work files found"

## Section 7: Work File Information

If a work file was created, let's examine its contents and statistics.

In [ ]:
import os

if os.path.exists(WORK_FILE):
    print("=" * 60)
    print("WORK FILE INFORMATION")
    print("=" * 60)
    print()
    !./kangaroo -winfo {WORK_FILE}
    print("\n" + "=" * 60)
else:
    print("⚠️  Work file not available for inspection.")

## Section 8: Performance Analysis

Calculate the performance metrics and estimate progress.

In [ ]:
import math

# Puzzle #120 parameters
range_bits = 119
expected_ops_bits = range_bits / 2 + math.log2(2.08)
expected_ops = 2 ** expected_ops_bits

print("=" * 60)
print("PUZZLE #120 STATISTICS")
print("=" * 60)
print(f"Range Width:           ~{range_bits} bits")
print(f"Expected Operations:   2^{expected_ops_bits:.2f} ({expected_ops:.2e})")
print(f"Square Root of Range:  2^{range_bits/2:.1f}")
print()
print("Estimated Time to Solve:")
print("  Single Tesla V100:   ~{:.0f} years".format(expected_ops / (2000e6 * 86400 * 365)))
print("  256× Tesla V100:     ~{:.0f} days".format(expected_ops / (500000e6 * 86400)))
print()
print("1-Hour Progress (estimated):")
# Assuming single Tesla V100 at 2000 MKey/s
ops_in_1_hour = 2000e6 * 3600
progress_percent = (ops_in_1_hour / expected_ops) * 100
print(f"  Operations in 1h:    {ops_in_1_hour:.2e}")
print(f"  Progress:            {progress_percent:.10f}%")
print()
print("ℹ️  Even 1 hour of work contributes to the overall search!")
print("ℹ️  Work files can be merged and resumed indefinitely.")
print("=" * 60)

## Section 9: Next Steps

### Resuming This Work

To continue from where you left off:

```bash
# Resume with the same configuration
./kangaroo -gpu -gpuId 0 -d 25 -ws -i puzzle120_work_YYYYMMDD_HHMMSS.work -w puzzle120_work_YYYYMMDD_HHMMSS.work -wi 300 -o result.txt
```

### Merging Work Files

If you have multiple work files from different runs:

```bash
# Merge work files (checks for collision)
./kangaroo -wm work1.work work2.work merged.work
```

### Distributed Solving

For distributed solving across multiple machines:

**Server:**
```bash
./kangaroo -s -d 25 -w server.work -wi 300 -o result.txt puzzle120.txt
```

**Clients:**
```bash
./kangaroo -gpu -gpuId 0 -w client.work -wi 600 -c server_ip
```

### Important Reminders

- 🔐 Always keep backups of your work files
- 💾 Work files from different machines can be merged
- ⚡ GPU performance varies based on model and settings
- 📊 Monitor memory usage with `-d` option if needed
- 🎯 Puzzle #120 is a long-term collaborative effort

## Section 10: Download Work File

If running in a cloud environment (like Google Colab or Kaggle), download your work file to continue later.

In [ ]:
import os

# Check if we're in a cloud environment
try:
    from google.colab import files
    is_colab = True
except:
    is_colab = False

if is_colab and os.path.exists(WORK_FILE):
    print(f"Downloading work file: {WORK_FILE}")
    files.download(WORK_FILE)
    print("✅ Download complete!")
elif os.path.exists(WORK_FILE):
    print(f"Work file location: {os.path.abspath(WORK_FILE)}")
    print("\nTo download manually, locate the file in your file browser.")
    print("Or use command: scp, rsync, or your cloud provider's tools.")
else:
    print("⚠️  No work file to download.")

---

## Summary

This notebook has:

✅ Verified GPU and CUDA availability  
✅ Compiled Kangaroo with GPU support  
✅ Configured Puzzle #120 parameters  
✅ Run GPU solver for exactly 1 hour  
✅ Saved work file for future resumption  
✅ Analyzed performance and progress  

### Educational Value

- Demonstrated GPU-accelerated ECDLP solving
- Showed work file management for long-running tasks
- Illustrated the computational difficulty of large puzzles
- Provided framework for distributed solving

### Disclaimer

⚠️ **For Educational and Research Purposes Only**

This tool demonstrates:
- Elliptic curve cryptography concepts
- GPU parallel computing optimization
- Discrete logarithm problem complexity
- Distributed computing architectures

Do not use this tool for unauthorized access to cryptocurrency wallets or keys you don't own.

---

**🦘 Happy Hunting! Remember: Solving puzzles requires patience, persistence, and often collaboration! 🦘**